In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV3Small

IMG_SIZE = 224

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
main_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'

train_dir = os.path.join(main_path, 'train')
valid_dir = os.path.join(main_path, 'valid')
test_dir = os.path.join(main_path, 'test')

In [3]:
train_data = tf.keras.utils.image_dataset_from_directory(train_dir, 
                                                         shuffle=True,
                                                         image_size=(IMG_SIZE,IMG_SIZE)).prefetch(1)
valid_data = tf.keras.utils.image_dataset_from_directory(valid_dir, 
                                                         shuffle=True,
                                                         image_size=(IMG_SIZE,IMG_SIZE)).prefetch(1)
test_data = tf.keras.utils.image_dataset_from_directory(test_dir, 
                                                         shuffle=True,
                                                         image_size=(IMG_SIZE,IMG_SIZE)).prefetch(1)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [19]:
inp = tf.keras.layers.Input([IMG_SIZE,IMG_SIZE,3], dtype=tf.float32)
x = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))(inp)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1000, activation='relu')(x)
output = layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inp, output, name='classification_model')

In [20]:
model.summary()

Model: "classification_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 global_average_pooling2d_2   (None, 576)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 1000)              577000    
                                                                 
 dense_7 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 1,517,121
Trainable params: 1,50

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
callbacks = [es, reduce_lr]

In [23]:
# history = model.fit(train_data, epochs=12, validation_data=valid_data, callbacks=callbacks)


Epoch 1/12
3125/3125 [==============================] - 466s 141ms/step - loss: 0.1093 - accuracy: 0.9575 - val_loss: 0.3474 - val_accuracy: 0.9094 - lr: 0.0010
Epoch 2/12
3125/3125 [==============================] - 263s 84ms/step - loss: 0.0434 - accuracy: 0.9843 - val_loss: 7.3203 - val_accuracy: 0.5715 - lr: 0.0010
Epoch 3/12
3125/3125 [==============================] - 239s 77ms/step - loss: 0.0294 - accuracy: 0.9897 - val_loss: 0.5390 - val_accuracy: 0.8814 - lr: 0.0010
Epoch 4/12
3125/3125 [==============================] - 238s 76ms/step - loss: 0.0240 - accuracy: 0.9915 - val_loss: 1.2260 - val_accuracy: 0.7910 - lr: 0.0010
Epoch 5/12
3125/3125 [==============================] - 248s 79ms/step - loss: 0.0230 - accuracy: 0.9918 - val_loss: 0.4711 - val_accuracy: 0.9259 - lr: 0.0010
Epoch 6/12
3125/3125 [==============================] - 239s 76ms/step - loss: 0.0151 - accuracy: 0.9946 - val_loss: 1.7225 - val_accuracy: 0.8253 - lr: 0.0010
Epoch 7/12
3125/3125 [=================

In [4]:
model = tf.keras.saving.load_model('/kaggle/input/mobilenet-fakeface/mobilenet_fakeface')
# tf.keras.saving.save_model(model, '/kaggle/working/mobilenet_fakeface', save_format='h5')

In [9]:
model.evaluate(test_data)

625/625 [==============================] - 171s 273ms/step - loss: 0.0133 - accuracy: 0.9965


[0.013328004628419876, 0.9965000152587891]

In [5]:
hard_test = pd.read_csv('/kaggle/input/hardfakevsrealfaces/data.csv')
hard_test.replace({'fake':0, 'real':1}, inplace=True)
hard_test.label.value_counts()

0    700
1    589
Name: label, dtype: int64

In [6]:
hard_path = '/kaggle/input/hardfakevsrealfaces'

hard = hard_test[hard_test['label']==0].sample(589)
hard = hard.append(hard_test[hard_test['label']==1])

/tmp/ipykernel_32/3442927800.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hard = hard.append(hard_test[hard_test['label']==1])


In [7]:
hard_dataset = tf.keras.utils.image_dataset_from_directory(hard_path,
                                                          image_size = (224,224)).prefetch(1)

Found 1289 files belonging to 2 classes.


In [8]:
model.evaluate(hard_dataset)

41/41 [==============================] - 13s 259ms/step - loss: 4.5751 - accuracy: 0.6587


[4.575099468231201, 0.6586501002311707]